In [ ]:
# commonset += f"Name: {self.name}\n"
# commonset += f"Age: {self.age}\n"
# commonset += f"Innate traits: {self.innate}\n"
# commonset += f"Learned traits: {self.learned}\n"
# commonset += f"Currently: {self.currently}\n"
# commonset += f"Lifestyle: {self.lifestyle}\n"
# commonset += f"Daily plan requirement: {self.daily_plan_req}\n"
# commonset += f"Current Date: {self.curr_time.strftime('%A %B %d')}\n"

In [ ]:
# prompt_input = []
# prompt_input += [persona.scratch.get_str_iss()]
# prompt_input += [persona.scratch.get_str_lifestyle()]
# prompt_input += [persona.scratch.get_str_curr_date_str()]
# prompt_input += [persona.scratch.get_str_firstname()]
# prompt_input += [f"{str(wake_up_hour)}:00 am"]

In [ ]:
# daily_planning_v6.txt

# Variables (this section up to commentblockmarker isn't used): 
# !<INPUT 0>! -- Commonset
# !<INPUT 1>! -- Lifestyle
# !<INPUT 2>! -- Reverie date time now
# !<INPUT 3>! -- Persona first names
# !<INPUT 4>! -- wake_up_hour

# <commentblockmarker>###</commentblockmarker>
# !<INPUT 0>!

# In general, !<INPUT 1>!
# Today is !<INPUT 2>!. Here is !<INPUT 3>!'s plan today in broad-strokes (with the time of the day. e.g., have a lunch at 12:00 pm, watch TV from 7 to 8 pm): 1) wake up and complete the morning routine at !<INPUT 4>!, 2)

In [1]:
fs = ['1) wake up and complete the morning routine at 6:00 am', 
        '2) eat breakfast at 7:00 am', 
        '3) read a book from 8:00 am to 12:00 pm', 
        '4) have lunch at 12:00 pm', 
        '5) take a nap from 1:00 pm to 4:00 pm', 
        '6) relax and watch TV from 7:00 pm to 8:00 pm', 
        '7) go to bed at 11:00 pm'] 

In [ ]:
# daily_planning_v6

gpt_param = {"engine": "text-davinci-003", "max_tokens": 500, 
            "temperature": 1, "top_p": 1, "stream": False,
            "frequency_penalty": 0, "presence_penalty": 0, "stop": None}

In [ ]:
# def __func_clean_up(gpt_response, prompt=""):
# cr = []
# _cr = gpt_response.split(")")
# for i in _cr: 
#     if i[-1].isdigit(): 
#     i = i[:-1].strip()
#     if i[-1] == "." or i[-1] == ",": 
#         cr += [i[:-1].strip()]
# return cr

# example output response cleaned up
#
# fs = ['wake up and complete the morning routine at 6:00 am', 
#         'eat breakfast at 7:00 am', 
#         'read a book from 8:00 am to 12:00 pm', 
#         'have lunch at 12:00 pm', 
#         'take a nap from 1:00 pm to 4:00 pm', 
#         'relax and watch TV from 7:00 pm to 8:00 pm', 
#         'go to bed at 11:00 pm'] 

# output = ([f"wake up and complete the morning routine at {wake_up_hour}:00 am"]
#             + output)

# persona.scratch.daily_req = generate_first_daily_plan(persona, 
#                                                         wake_up_hour)

In [ ]:
# focal_points = [f"{p_name}'s plan for {persona.scratch.get_str_curr_date_str()}.",
#                 f"Important recent events for {p_name}'s life."]
# retrieved = new_retrieve(persona, focal_points)

In [1]:
from datetime import datetime
import faiss

from langchain.embeddings.openai import OpenAIEmbeddings

from langchain.docstore import InMemoryDocstore
from langchain.vectorstores import FAISS

from langchain.retrievers import TimeWeightedVectorStoreRetriever
from langchain.chat_models import ChatOpenAI

from discussion_agents.memory.base_memory import GenerativeAgentMemory
from discussion_agents.model.base_agent import GenerativeAgent

import dotenv
import os

dotenv.load_dotenv("../.env")
openai_api_key = os.getenv("OPENAI_API_KEY")

def create_memory_retriever():
    embeddings_model = OpenAIEmbeddings(openai_api_key=openai_api_key)
    embedding_size = 1536
    index = faiss.IndexFlatL2(embedding_size)
    vectorstore = FAISS(embeddings_model.embed_query, index, InMemoryDocstore({}), {})
    retriever = TimeWeightedVectorStoreRetriever(
        vectorstore=vectorstore,
        otherScoreKeys=["importance"],
        k=15
    )
    return retriever

USER_NAME = "Person A"  # The name you want to use when interviewing the agent.
LLM = ChatOpenAI(
    openai_api_key=openai_api_key,
    model_name="gpt-3.5-turbo",
    max_tokens=1500,
    temperature=1
)

tommies_memory = GenerativeAgentMemory(
    llm=LLM,
    memory_retriever=create_memory_retriever(),
    verbose=False,
    reflection_threshold=8,
)

tommie = GenerativeAgent(
    name="Tommie",
    age=25,
    innate_traits="anxious, likes design, talkative",
    learned_traits="Tommie is currently attending University of College and he's studying cognitive science and user design.",
    lifestyle="Tommie wakes up around 8 am, sleeps around 10 pm, and eats dinner around 7 pm.",
    status="Tommie is currently setting up a podcast to chat about the latest user design tricks.",
    daily_req=[],
    daily_plan_req="Tommie always has to eat 3 meals a day.",
    f_daily_schedule=[],
    memory_retriever=create_memory_retriever(),
    llm=LLM,
    memory=tommies_memory
)

In [2]:
# from datetime import datetime

# tommie.generate_daily_req(current_day=datetime.now())

# tommie.daily_req

# tommie.update_status_and_daily_plan_req(current_day=datetime.now())

# tommie.daily_plan_req
# tommie.status

In [3]:
# !<INPUT 0>! -- Schedule format
# !<INPUT 1>! -- Commonset
# !<INPUT 2>! -- prior_schedule
# !<INPUT 3>! -- intermission_str
# !<INPUT 4>! -- intermission 2
# !<INPUT 5>! -- prompt_ending

# Hourly schedule format: 
# !<INPUT 0>!
# ===
# !<INPUT 1>!
# !<INPUT 2>!
# !<INPUT 3>!!<INPUT 4>!
# !<INPUT 5>!

In [4]:
tommie.generate_hourly_schedule_top_3(current_day=datetime.now())

[['sleeping', 480],
 ['waking up and getting ready for the day', 60],
 ['attending classes at University of College', 60],
 ['attending a lecture on cognitive science', 60],
 ['attending a workshop on user design', 60],
 ['having lunch', 60],
 ['continuing classes at University of College', 420],
 ['setting up a podcast to chat about the latest user design tricks', 180],
 ['sleeping', 60]]